<div style="height:300;background-color:#021A36;color:white;border-radius: 15px;padding:10px">
    <h1>Logistic regression on: ID_2 and ID_44</h1>
    <h1>Train prediction on: ID_44</h1>
    <h2></h2>
    <hr>
    <h3>By: Gari Ciodaro Guerra</h3>
</div>

In [1]:
from fastai.vision import *
from fastai.metrics import *
from fastai.callbacks import *
import torchvision.models as models
from fastai.callbacks.hooks import *
import imageio
from scipy.stats import ttest_ind,bartlett
import mne
import seaborn as sns

#AUxiliar class to load pickle objects from disk
path_to_class="/home/gari/Desktop/master_tesis_v3/MLDiagnosisTool/Classes/"
sys.path.append(path_to_class)
from OFHandlers import OFHandlers as OFH


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.metrics import (roc_auc_score,
                             precision_score,
                             recall_score,
                             f1_score,
                             average_precision_score,
                             accuracy_score,
                             balanced_accuracy_score,
                             roc_curve)

#auxiliar dictionary of models from pytorch
dic_models={"resnet18":models.resnet18,
            "resnet50":models.resnet50,
            "alexnet":models.alexnet,
            "squeezenet1_0":models.squeezenet1_0
            }

In [2]:
# path where the experiment is stored (trained model)
experiment="alpha_beta"
path="/home/gari/Desktop/master_tesis_v3/Data/Datasets_image_"+experiment

In [3]:
# general configurations
string_model="resnet18"
bs=50
pretrained=True
ps=0.5
unfreeze_net=True
number_epochs=10
track_save_callback=False

In [4]:
data = ImageDataBunch.from_folder(path,bs=bs).normalize(imagenet_stats)

/home/gari/anaconda3/envs/fast_ai/lib/python3.7/site-packages/fastai/data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


In [5]:
#create the object "learn", the same way as in training.
learn = cnn_learner(data,dic_models.get(string_model),
                        metrics=[accuracy, AUROC()],
                        pretrained=pretrained,
                        ps=ps)

In [6]:
#load the trained weights
learn.load("stage-1")

Learner(data=ImageDataBunch;

Train: LabelList (995 items)
x: ImageList
Image (3, 380, 390),Image (3, 380, 390),Image (3, 380, 390),Image (3, 380, 390),Image (3, 380, 390)
y: CategoryList
Healthy,Healthy,Healthy,Healthy,Healthy
Path: /home/gari/Desktop/master_tesis_v3/Data/Datasets_image_alpha_beta;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: /home/gari/Desktop/master_tesis_v3/Data/Datasets_image_alpha_beta;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [22]:
import os
train_images=os.listdir(path+"/train_N/")


7

In [14]:
path_identity="/home/gari/Desktop/master_tesis_v3/Data/IdDataSet/"
df_train_identity=OFH.load_object(path_identity+"df_train_identity.file")

In [43]:
prediction=[]
indexes=[]
train_images=os.listdir(path+"/train_N/")
for each_image in train_images:
    img = open_image(path+"/train_N/"+each_image)
    p=learn.predict(img)
    tensor = p[2]
    single_pred=np.round(tensor.cpu().detach().numpy()[0], 5)
    prediction.append(single_pred)
    indexes.append(int(each_image.split(".")[0]))
pred_data=pd.DataFrame(prediction,columns=["Predicted_Target"],index=indexes)


In [46]:
predictions=pred_data

In [47]:
df_train_id_44=predictions.rename(columns={"Predicted_Target":"id_44_predictions"},inplace=False)
df_train_id_44

,id_44_predictions
7,0.99985
186,0.99997
458,0.00140
17,0.99709
954,0.99999
...,...
962,0.99999
432,0.00000
108,0.99967
779,0.99957


In [48]:
OFH.save_object("./combined/df_train_id_44.file",df_train_id_44)